In [1]:
import pandas as pd 
import numpy as np 
from datetime import date
import requests
import json

In [2]:
def buscarMarca():
    url = "https://veiculos.fipe.org.br/api/veiculos//ConsultarMarcas?codigoTabelaReferencia=267&codigoTipoVeiculo=1"

    payload = "{\r\n    \r\n}"
    headers = {
    'Content-Type': 'text/plain',
    'Cookie': 'ROUTEID=.3'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return pd.DataFrame(json.loads(response.content))

def buscarCarros(marcas):
    modelos = pd.DataFrame()
    for marca in marcas:
        url = f"https://veiculos.fipe.org.br/api/veiculos//ConsultarModelos?codigoTipoVeiculo=1&codigoTabelaReferencia=267&codigoMarca={marca}"
        payload = "{\r\n    \r\n}"
        headers = {
        'Content-Type': 'text/plain',
        'Cookie': 'ROUTEID=.3'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        try:
            veiculos = json.loads(response.content)
        except:
            print('')
        df_temp = pd.DataFrame(veiculos['Modelos'])
        df_temp['id_marca'] = f"{marca}"
        modelos = pd.concat([df_temp, modelos])
    return modelos

marcas_df = buscarMarca()
marcas_df.columns = ['nome_marca', 'id_marca']
marcas_df

modelos_df = buscarCarros(marcas_df['id_marca'])


In [3]:
df = pd.DataFrame(modelos_df)
df.columns = ['nome_modelo', 'id_modelo', 'id_marca']
df

,nome_modelo,id_modelo,id_marca
0,Buggy Walk Sport 1.6 8V 58cv,3998,120
0,Way 1.6 Total Flex 8V Mec.,5088,163
1,Way 1.8 Total Flex 8V Mec.,5089,163
0,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x2 Die,5585,59
1,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x4 Die,5586,59
...,...,...,...
6,MARRUÁ AM 200 2.8 CS TDI Diesel,4568,2
7,MARRUÁ AM 50 2.8 140cv TDI Diesel,4569,2
0,Integra GS 1.8,1,1
1,Legend 3.2/3.5,2,1


In [4]:
dataset_final = marcas_df.merge(modelos_df, how='inner')

dataset_final.to_csv('output/scrapping_marcas_modelos_fipe.csv')